In [ ]:
!pip install stanza
!pip install nltk

In [ ]:
from string import punctuation
import re
import csv
import sys
import stanza
import nltk
nltk.download('punkt')

Открываю файл text.csv, немного чищу и разделяю на предложения, сохраняю в sentences.

In [ ]:
def clear(line):  # эта функция хорошо работает только с этим текстом, потому что я его рассмотрела, и нашла, что убрать
    clear_txt = re.sub("\* .*(\(Примеч\. Лермонтова\.\)|Ред\.)", "", line)  # убираем Примечания, потому что они посреди других предложений находятся
    clear_txt = re.sub('[^С*]\*', "", clear_txt) # убираем звёздочки от примечаний, оставляем один случай "майор С***" (превратится в)
    clear_txt = re.sub('Г<осподин>', 'Господин', clear_txt)
    return clear_txt

In [ ]:
csv.field_size_limit(sys.maxsize)
sentences = []
with open("crawler_text.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        clear_txt = clear(row[0])
        sentences.extend([(i.strip(), row[1]) for i in nltk.sent_tokenize(clear_txt)])

In [ ]:
sentences2 = []
with open("crawler_text.csv") as f:
    reader2 = csv.reader(f)
    for row in reader2:
        clear_txt2 = clear(row[0])
        sentences2.extend([(i.strip(), row[1]) for i in nltk.sent_tokenize(clear_txt2)])

In [ ]:
sentences2.pop(0) # там строка заголовка

('Text', 'Part')

In [ ]:
for i in range(len(sentences)):
    if sentences[i][0] != sentences2[i][0]:
        print(sentences[i][0], sentences2[i][0])

Определяю морфологию с помощью stanza.
Сохраняю данный в следующем формате для поиска:

Мама мыла раму., (мама; мыла; раму), (мама; мыть; рама), (Noun; Verb; Noun), Про мытьё рамы. Глава 1

(Скобки и пробелы только чтобы было лучше понятно здесь, их не будет в файле)

In [ ]:
sentences.pop(0) # там строка заголовка

('Text', 'Part')

In [ ]:
punctuation += '«»—'

In [ ]:
ppln = stanza.Pipeline('ru', processors='tokenize,pos,lemma')
rows = []
for sent, chapter in sentences:
    words = []
    lemmas = []
    pos = []
    doc = ppln(sent)
    for snt in doc.sentences:
        for word in snt.words:
            if word.upos != 'PUNCT':
                words.append(word.text)
                lemmas.append(word.lemma)
                pos.append(word.upos)
    words_line = ";".join(words)
    lemmas_line = ';'.join(lemmas)
    pos_line = ';'.join(pos)
    rows.append([sent, words_line, lemmas_line, pos_line, chapter])


In [ ]:
with open("corpus_data.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Sentence', 'Tokens', 'Lemmas', 'POS', 'Chapter'])
    writer.writerows(rows)